In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
#Lectura de datos y construcción del dataframe principal
empresas = {
    "Bimbo": "BIMBOA.MX",
    "Cemex": "CEMEXCPO.MX",
    "Alfa SA A": "ALFAA.MX",
    "Alsea SA": "ALSEA.MX",
    "America Movil SAB de CV B": "AMXB.MX",
    "Arca Continental SAB de CV": "AC.MX",
    "Banco del Bajio SA": "BBAJIOO.MX",
    "Becle SA De CV": "CUERVO.MX",
    "Bolsa Mexicana de Valores SA de CV": "BOLSAA.MX",
    "Coca Cola Femsa SAB de CV UBL": "KOFUBL.MX",
    "Corporacion Inmobiliaria Vesta SAB de CV": "VESTA.MX",
    "El Puerto de Liverpool SAB de CV": "LIVEPOLC-1.MX",
    "Fomento Economico Mexicano SAB de CV": "FEMSAUBD.MX",
    "Genomma Lab Internacional SA de CV": "LABB.MX",
    "Gentera SAB de CV": "GENTERA.MX",
    "Gruma SAB B": "GRUMAB.MX",
    "Grupo Aeroportuario del Centro Norte SAB de CV": "OMAB.MX",
    "Grupo Aeroportuario del Sureste SAB de CV B": "ASURB.MX",
    "Grupo Carso SAB de CV": "GCARSOA1.MX",
    "Grupo Cementos de Chihuahua SAB de CV": "GCC.MX",
    "Grupo Comercial Chedraui SA de CV": "CHDRAUIB.MX",
    "Grupo Financiero Banorte O": "GFNORTEO.MX",
    "Grupo Financiero Inbursa O": "GFINBURO.MX",
    "Grupo Mexico SAB de CV B": "GMEXICOB.MX",
    "Grupo Televisa SAB CPO": "TLEVISACPO.MX",
    "Industrias Peñoles": "PE&OLES.MX",
    "Kimberly Clark de Mexico SAB de CV A": "KIMBERA.MX",
    "La Comer SAB de CV UBC": "LACOMERUBC.MX",
    "Megacable Holdings SAB de CV": "MEGACPO.MX",
    "Orbia Advance Corporation SAB de CV": "ORBIA.MX",
    "Qualitas Controladora SAB de CV": "Q.MX",
    "Regional SA de CV": "RA.MX",
    "Walmart de Mexico SAB de CV": "WALMEX.MX"
}
start_date = '2015-01-01'
end_date = '2024-12-31'
paro=1
for nombre, ticker in empresas.items():
    df1 =pd.read_csv(f"{ticker}_{start_date}_to_{end_date}.csv")
    df1 = df1.drop([0, 1])
    df1 = df1.drop(['Price', "Open"], axis=1)
    df1.columns = [f"{nombre}_Close", f"{nombre}_High", f"{nombre}_Low", f"{nombre}_Volume"]
    if paro == 1:
        df2=df1
    else:
        df2=pd.concat([df1, df2], axis=1)
    df2.dropna(axis=1)
    paro =2
df3=df2
df_pivot=pd.read_csv("AC.MX_2015-01-01_to_2024-12-31.csv")
df_pivot = df_pivot.drop([0, 1])
df_pivot = df_pivot.drop(["Open","Close","High","Low","Volume"], axis=1)
df2=pd.concat([df_pivot,df2], axis=1)
df2.to_csv('Usable.csv', header=True)

In [4]:
semestres=["02/01/2015","30/06/2015","31/12/2015","30/06/2016", "30/12/2016", "30/06/2017", "29/12/2017", "29/06/2018", "31/12/2018", "28/06/2019", "31/12/2019", "30/12/2020", "31/12/2020", "30/12/2021","31/12/2021", "30/06/2022", "30/12/2022", "30/06/2023", "29/12/2023","28/06/2024", "30/12/2024"]
df = pd.read_csv('Usable.csv', parse_dates=['Price'])
df.set_index('Price', inplace=True)
semestresIPC=["1/01/2015","1/06/2015","1/12/2015","1/01/2016","1/06/2016","1/12/2016","1/01/2017","1/06/2017","1/12/2017","1/01/2018","1/06/2018","1/12/2018","1/01/2019","1/06/2019","1/12/2019","1/01/2020","1/06/2020","1/12/2020","1/01/2021","1/06/2021","1/12/2021","1/01/2022","1/06/2022","1/12/2022","1/01/2023","1/06/2023","1/12/2023","1/01/2024","1/06/2024","1/12/2024"]
dfipc = pd.read_csv('IPC.csv', parse_dates=['Fecha'])
dfipc.set_index('Fecha', inplace=True)

In [22]:
MarketT=0
marketL=[]
VolaT=0
volaL=[]
for p in range(len(semestres)-1):
    date_range = df.loc[semestres[p]:semestres[p+1]]
    marketL.append([])
    volaL.append([])
    for nombre, ticker in empresas.items():
        date_range[f"{nombre}_Volume"] = pd.to_numeric(date_range[f"{nombre}_Volume"], errors='coerce')
        date_range[f"{nombre}_Close"] = pd.to_numeric(date_range[f"{nombre}_Close"], errors='coerce')
        date_range[f"{nombre}_MCap"]=date_range[f"{nombre}_Volume"]*date_range[f"{nombre}_Close"]
        marketcap = date_range[f"{nombre}_MCap"].mean()
        marketL[p].append(marketcap)
        date_range[f"{nombre}_High"] = pd.to_numeric(date_range[f'{nombre}_High'], errors='coerce')
        date_range[f'{nombre}_Low'] = pd.to_numeric(date_range[f'{nombre}_Low'], errors='coerce')
        date_range[f"{nombre}_Desv"]=((date_range[f"{nombre}_High"]+date_range[f"{nombre}_Low"])/2)
        if ((date_range[f"{nombre}_Desv"]).std())==0:
            volaL[p].append(0)
        else:
            desviacion = 1/((date_range[f"{nombre}_Desv"]).std())
            volaL[p].append(desviacion)
propMark=[]
propVola=[]
MC_Total=[]
Vola_Total=[]
for i in range(len(semestres)-1):
    propMark.append([])
    propVola.append([])
    MC_Total.append([])
    Vola_Total.append([])
    for j in range(len(empresas)):
        MarketT=MarketT+marketL[i][j]
        VolaT=VolaT+volaL[i][j]
        if j==(len(empresas)-1):
            for k in range(len(empresas)):
                propVola[i].append((volaL[i][k])/VolaT)
                propMark[i].append((marketL[i][k])/MarketT)
            MC_Total[i]=MarketT
            Vola_Total[i]=VolaT
            VolaT=0
            MarketT=0

C:\Users\praxy\AppData\Local\Temp\ipykernel_15884\2538076068.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  date_range[f"{nombre}_Volume"] = pd.to_numeric(date_range[f"{nombre}_Volume"], errors='coerce')
C:\Users\praxy\AppData\Local\Temp\ipykernel_15884\2538076068.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  date_range[f"{nombre}_Close"] = pd.to_numeric(date_range[f"{nombre}_Close"], errors='coerce')
C:\Users\praxy\AppData\Local\Temp\ipykernel_15884\2538076068.py:12: SettingWithCopyWarning: 
A

In [15]:
C1_2={'Grupo Financiero Inbursa O': 22, "Bimbo":0,
    "Cemex":1,"Alfa SA A":2,"Arca Continental SAB de CV":5,
    "Banco del Bajio SA":6,"Becle SA De CV": 7,"Coca Cola Femsa SAB de CV UBL":9,
    "Corporacion Inmobiliaria Vesta SAB de CV": 10,
    "El Puerto de Liverpool SAB de CV": 11,
    "Fomento Economico Mexicano SAB de CV": 12,
    "Genomma Lab Internacional SA de CV": 13,
    "Gentera SAB de CV": 14,
    "Gruma SAB B":15,
    "Grupo Aeroportuario del Centro Norte SAB de CV": 16,
    "Grupo Aeroportuario del Sureste SAB de CV B": 17,"Grupo Cementos de Chihuahua SAB de CV": 19,
    "Grupo Comercial Chedraui SA de CV": 20, "Grupo Mexico SAB de CV B": 23,
    "Grupo Televisa SAB CPO": 24,
    "Industrias Peñoles": 25,
    "Kimberly Clark de Mexico SAB de CV A":26,
    "La Comer SAB de CV UBC": 27,
    "Megacable Holdings SAB de CV":28}
C2_2={'Bolsa Mexicana de Valores SA de CV': 8,
    "Alsea SA": 3,
    "America Movil SAB de CV B": 4,
    "Grupo Carso SAB de CV": 18,
    "Grupo Financiero Banorte O": 21,
    "Megacable Holdings SAB de CV": 28,
    "Orbia Advance Corporation SAB de CV": 29,
    "Qualitas Controladora SAB de CV": 30,
    "Regional SA de CV": 31,
    "Walmart de Mexico SAB de CV": 32}
C1_4={"Becle SA De CV": 7,
      "Alsea SA": 3,
      "Grupo Cementos de Chihuahua SAB de CV":19,
      "Regional SA de CV": 31}
C2_4={"Grupo Financiero Banorte O": 21,
      'Bolsa Mexicana de Valores SA de CV': 8,
      "Grupo Carso SAB de CV": 18,
      "Orbia Advance Corporation SAB de CV": 29,
      "Qualitas Controladora SAB de CV": 30}
C3_4={"Grupo Mexico SAB de CV B": 23,
      "Bimbo": 0,
      "Alfa SA A": 2,
      "America Movil SAB de CV B": 4,
      "Arca Continental SAB de CV": 5,
      'Grupo Financiero Inbursa O': 22,
      "Grupo Televisa SAB CPO": 24,
      "Industrias Peñoles": 25,
      "Kimberly Clark de Mexico SAB de CV A": 26}
C4_4={"La Comer SAB de CV UBC": 27,
      "Cemex": 1,
      "Banco del Bajio SA": 6,
      "Coca Cola Femsa SAB de CV UBL": 9,
      "Corporacion Inmobiliaria Vesta SAB de CV":10,
      "El Puerto de Liverpool SAB de CV": 11,
    "Fomento Economico Mexicano SAB de CV": 12,
    "Genomma Lab Internacional SA de CV": 13,
    "Gentera SAB de CV":14,
    "Gruma SAB B": 15,
    "Grupo Aeroportuario del Centro Norte SAB de CV": 16,
    "Grupo Aeroportuario del Sureste SAB de CV B": 7,
    "Grupo Comercial Chedraui SA de CV": 20,
    "Megacable Holdings SAB de CV": 28,
    "Walmart de Mexico SAB de CV": 32}
C1_8={"Alsea SA": 3}
C2_8={"America Movil SAB de CV B":4}
C3_8={"Grupo Cementos de Chihuahua SAB de CV":19}
C4_8={"Grupo Financiero Banorte O": 21,
      'Bolsa Mexicana de Valores SA de CV': 8,
      "Grupo Carso SAB de CV": 18,
      "Industrias Peñoles": 25,
      "Orbia Advance Corporation SAB de CV": 29,
      "Qualitas Controladora SAB de CV":30}
C5_8={'Grupo Financiero Inbursa O': 22,
      "Alfa SA A": 2,
      "Banco del Bajio SA": 6,
      "Coca Cola Femsa SAB de CV UBL": 9,
      "Genomma Lab Internacional SA de CV":13,
      "Grupo Aeroportuario del Centro Norte SAB de CV":16,
      "Kimberly Clark de Mexico SAB de CV A":26}
C6_8={"Grupo Mexico SAB de CV B":23,
    "Bimbo":0,
    "Arca Continental SAB de CV":5,
    "Grupo Televisa SAB CPO":24}
C7_8={"La Comer SAB de CV UBC": 27,
      "Cemex": 1,
      "Corporacion Inmobiliaria Vesta SAB de CV":10,
      "El Puerto de Liverpool SAB de CV":11,
      "Fomento Economico Mexicano SAB de CV":12,
      "Gentera SAB de CV":14,
      "Gruma SAB B":15,
      "Grupo Comercial Chedraui SA de CV":20,
      "Megacable Holdings SAB de CV":28,
      "Walmart de Mexico SAB de CV":32}
C8_8={"Regional SA de CV":31,
      "Becle SA De CV":7}

In [71]:
Asignacion_market=[]
Asignacion_vola=[]
Asignacion_EW=[]
MarketT=0
VolaT=0
#Para 2 clústeres
for j in range(len(semestres)-1):
    for nombre, id in C1_2.items():
        MarketT=marketL[j][id]+MarketT
        VolaT=volaL[j][id]+VolaT
    Asignacion_market.append(MarketT/MC_Total[j])
    Asignacion_vola.append(VolaT/Vola_Total[j])


In [ ]:
def asignacion_de_accion_representativa(cluster, x, y):
    Asignacion_market=[]
    #Asignacion_vola=[]
    Asignacion_EW=[]
    MarketT=0
    #VolaT=0
    aux=0
    for j in range(len(semestres)-1):
        for nombre, id in cluster.items():
            MarketT=marketL[j][id]+MarketT
            #VolaT=propVola[j][id]+VolaT
        Asignacion_market.append(MarketT/MC_Total[j])
        #Asignacion_vola.append(VolaT/Vola_Total[j])
        aux=((x/y)/(1))
        Asignacion_EW.append(aux)
        MarketT=0
        #VolaT=0
    return Asignacion_market #,Asignacion_vola, Asignacion_EW
    return Asignacion_vola
    return Asignacion_EW


In [94]:
asignacion_de_accion_representativa(C1_2,len(C1_2),len(empresas))

([0.7048460681399027,
  0.739785219950799,
  0.7045613114622882,
  0.7275596296453131,
  0.6902949982753526,
  0.6772193215813023,
  0.6453987763595372,
  0.6138720009989318,
  0.6175826118047398,
  0.6001545917972938,
  0.6142803619625555,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan],
 [0.027112428591608995,
  0.03716633417929755,
  0.04142238226480331,
  0.038502185397579936,
  0.03863609908626812,
  0.033254800679637375,
  0.038733721290957424,
  0.05745107646652516,
  0.04155872194045451,
  0.048608089800270864,
  0.07262340595143034,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan],
 [0.7272727272727273,
  0.7272727272727273,
  0.7272727272727273,
  0.7272727272727273,
  0.7272727272727273,
  0.7272727272727273,
  0.7272727272727273,
  0.7272727272727273,
  0.7272727272727273,
  0.7272727272727273,
  0.7272727272727273,
  0.7272727272727273,
  0.7272727272727273,
  0.7272727272727273,
  0.7272727272727273,
  0.7272727272727273,
  0.7272727272727273,

In [106]:
asignacion_de_accion_representativa(C2_2,len(C2_2),len(empresas))

([0.3027090135298159,
  0.2664737198114911,
  0.30354353784914245,
  0.27999919164827997,
  0.3181294760788251,
  0.33469446928057744,
  0.36351755401396657,
  0.4115106448699739,
  0.4016221093059045,
  0.4142695186325479,
  0.3977659213675579,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan],
 [0.012952165214504712,
  0.0160441810568988,
  0.018741142409642676,
  0.023389580647813005,
  0.016188394566811676,
  0.015259092485925227,
  0.019946157987272357,
  0.019792204795917742,
  0.01872292412071791,
  0.01830919442983281,
  0.03774863307329284,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan],
 [0.30303030303030304,
  0.30303030303030304,
  0.30303030303030304,
  0.30303030303030304,
  0.30303030303030304,
  0.30303030303030304,
  0.30303030303030304,
  0.30303030303030304,
  0.30303030303030304,
  0.30303030303030304,
  0.30303030303030304,
  0.30303030303030304,
  0.30303030303030304,
  0.30303030303030304,
  0.30303030303030304,
  0.30303030303030304,

In [104]:
auxk=0
aux2=0
j=0
errores=[]
erroresdesv=[]
for i in range(len(semestres)-1):
    errores.append([])
    erroresdesv.append([])
    dfpruebaf = df.loc[semestres[i]:semestres[i+1]]
    dfipc_mes=dfipc.loc[semestresIPC[i+j]:semestresIPC[i+j+1]]
    for nombre, id in C1_2.items():
        dfpruebaf[f"{nombre}_Close"]=dfpruebaf[f"{nombre}_Close"].mul(propMark[i][aux2]*len(C1_2))
        if aux2 ==0:
            dfpruebaf["Cierre"]=dfpruebaf[f"{nombre}_Close"]
        else:
            dfpruebaf["Cierre"]=dfpruebaf[f"{nombre}_Close"]+dfpruebaf["Cierre"]
        aux2=aux2+1
    aux2=0
    dfprueba_mes= dfpruebaf.resample('M').mean()
    #dferror=pd.concat([dfprueba_mes,dfipc_mes["IPC"]], axis=1)
       # correlation = dfprueba_mes[f"{nombre}_Close"].corr(dfipc_mes['IPC'])
    correlation = pd.Series(dfprueba_mes[f"Cierre"].values).corr(pd.Series(dfipc_mes['IPC'].values))
    desv=(pd.Series(dfprueba_mes[f"Cierre"].values)-(pd.Series(dfipc_mes['IPC'].values))).std()
    erroresdesv[i].append((desv))
    if correlation<0:
        errores[i].append(1+correlation)
    else:
        errores[i].append(correlation)
    auxk=auxk+1
    if auxk==2:
        j=j+1
        auxk=0

C:\Users\praxy\AppData\Local\Temp\ipykernel_15884\3754383450.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfpruebaf[f"{nombre}_Close"]=dfpruebaf[f"{nombre}_Close"].mul(propMark[i][aux2]*len(C1_2))
C:\Users\praxy\AppData\Local\Temp\ipykernel_15884\3754383450.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfpruebaf["Cierre"]=dfpruebaf[f"{nombre}_Close"]
C:\Users\praxy\AppData\Local\Temp\ipykernel_15884\3754383450.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

In [ ]:
import gurobipy as gb
from gurobipy import GRB
k=2 #4 y 8
repeticion=3
for p in range(repeticion):
    m=gb.Model("Cluster")
    desicion_accion={}
    aux1=0
    for nombre1 in empresas.items():
        aux2=0
        for nombre2 in empresas.items():
            desicion_accion[aux1,aux2]=m.addVar(name=f"{nombre1}_{nombre2}", vtype=GRB.BINARY)
            aux2=aux2+1
        aux1=aux1+1
    accion_guardado={}
    aux1=0
    for nombre1 in empresas.items(): 
        accion_guardado[aux1]=m.addVar(name=f"{nombre1}_escogido", vtype=GRB.BINARY)
        aux1=aux1+1
    m.setObjective(gb.quicksum(df3_array[i][j]*desicion_accion[i,j] for i in range (len(empresas)) for j in range(len(empresas))), GRB.MAXIMIZE)
    for i in range (len(empresas)):
        m.addConstr(gb.quicksum(desicion_accion[i,j] for j in range (len(empresas)))==1)
    m.addConstr(gb.quicksum(accion_guardado[j] for j in range (len(empresas)))==k)
    for i in range (len(empresas)):
        for j in range (len (empresas)):
                m.addConstr(desicion_accion[i,j]<=accion_guardado[j])
    k=k*2
    m.optimize()
    m.write("prueba_op_cluster"+str(p+1)+".lp")
    g=open("resultados_"+str(p+1)+".txt","w")
    if m.ObjVal !=float("-inf"):
        for v in m.getVars():
            if v.x>0:
                print("%s %g" %(v.VarName, v.x))
                g.write("%s %g" %(v.VarName, v.x))
                g.write("\n")
        print("Obj: %g" % m.ObjVal)
        g.write("Obj: %g" % m.ObjVal)
        print(" Finished")
        print(" ")
        print(" ")
    g.close() 